In [1]:
import re
import yaml

In [2]:
def compilePath(child, parentPath=''):
    """
    Generates the query paths for a given child and all its children

    >>> child = {"id": "work", "type": "crm:E36_Visual_Item", "query": "$subject crm:P128_carries ?value .", "children": [{"id": "work_creation", "query": "$subject crm:P94i_was_created_by ?value .", "children" : [{"id": "work_creator", "query" : "$subject crm:P14_carried_out_by ?value ." }] }] }
    >>> print(compilePath(child, ""))
    $subject crm:P128_carries ?value_work .
    $subject crm:P128_carries/crm:P94i_was_created_by ?value_work_creation .
    $subject crm:P128_carries/crm:P94i_was_created_by/crm:P14_carried_out_by ?value_work_creator .
    """
    query = child['query']
    optional = child['optional'] if 'optional' in child else False
    
    subjectPathPattern = r'(?:\$subject\s)([^\s]*)'
    subjectPath = re.search(subjectPathPattern, query).group(1)
    
    completePath = parentPath + '/' + subjectPath if parentPath else subjectPath

    query = query.replace(subjectPath, completePath)
    
    # Namespace variables by prefixing them with (unique) field id
    query = re.sub(r'\?([^\s/,:,\-\\\(\)]*)', r'?\1_' + child['id'], query)
    
    if optional:
        query = "OPTIONAL { %s }\n" % query
    
    if 'children' in child:
        for c in child['children']:
            query = query + "\n" + compilePath(c, completePath)
    
    return query

In [9]:
def compileQuery(node):
    """
    >>> node = {"id": "artwork", "label": "Artwork", "type": "crm:E22_Human-Made_Object", "children": [{"id": "work", "type": "crm:E36_Visual_Item", "query": "$subject crm:P128_carries ?value .", "children": [{"id": "work_creation", "query": "$subject crm:P94i_was_created_by ?value .", "children" : [{"id": "work_creator", "optional": True, "query" : "$subject crm:P14_carried_out_by ?value ." }] }] }]}
    >>> print(compileQuery(node))
    SELECT * WHERE {
    $subject a crm:E22_Human-Made_Object
    $subject crm:P128_carries ?value_work .
    $subject crm:P128_carries/crm:P94i_was_created_by ?value_work_creation .
    OPTIONAL { $subject crm:P128_carries/crm:P94i_was_created_by/crm:P14_carried_out_by ?value_work_creator . }
    }
    """
    query = "SELECT * WHERE {\n"
    query = query + "$subject a " + node['type'] + "\n"
    for child in node['children']:
        query = query + compilePath(child)
        
    query = query+"}"
    return query

print(compileQuery(model[0]))

SELECT * WHERE {
$subject a crm:E22_Man-Made_Object
OPTIONAL {  $subject crm:P2_has_type ?value_artwork_genre . ?value_artwork_genre crm:P2_has_type sari:genre ; rdfs:label ?label_artwork_genre .  }
OPTIONAL {  $subject crm:P2_has_type ?value_artwork_medium . ?value_artwork_medium crm:P2_has_type sari:medium ; rdfs:label ?label_artwork_medium .  }
 $subject crm:P128_carries ?value_work . 
 $subject crm:P128_carries/crm:P94i_was_created_by ?value_work_creation . 
OPTIONAL {  $subject crm:P128_carries/crm:P94i_was_created_by/crm:P14_carried_out_by ?value_work_creator .  }

OPTIONAL {  $subject crm:P128_carries/crm:P94i_was_created_by/crm:P9_consists_of ?subcreation_work_creator_with_role . ?subcreation_work_creator_with_role crm:P2_has_type ?subcreation_type_work_creator_with_role ; crm:P14_carried_out_by ?subcreation_person_work_creator_with_role . ?subcreation_type_work_creator_with_role rdfs:label ?subcreation_type_label_work_creator_with_role . ?subcreation_person_work_creator_with_r

<generator object dict_generator at 0x7ff0f8340138>

In [4]:
def parseModelFromFile(inputFile):
    """
    Reads input model from filepath

    >>> model = parseModelFromFile('../models/bso.yml')
    >>> print(type(model))
    <class 'list'>
    """
    with open(inputFile, 'r') as f:
        modelData = yaml.safe_load(f.read())
    return modelData



In [6]:
inputFile = '../models/bso.yml'
model = parseModelFromFile(inputFile)

'SELECT * WHERE {\n$subject a crm:E22_Man-Made_Object\nOPTIONAL {  $subject crm:P2_has_type ?value_artwork_genre . ?value_artwork_genre crm:P2_has_type sari:genre ; rdfs:label ?label_artwork_genre .  }\nOPTIONAL {  $subject crm:P2_has_type ?value_artwork_medium . ?value_artwork_medium crm:P2_has_type sari:medium ; rdfs:label ?label_artwork_medium .  }\n $subject crm:P128_carries ?value_work . \n $subject crm:P128_carries/crm:P94i_was_created_by ?value_work_creation . \nOPTIONAL {  $subject crm:P128_carries/crm:P94i_was_created_by/crm:P14_carried_out_by ?value_work_creator .  }\n\nOPTIONAL {  $subject crm:P128_carries/crm:P94i_was_created_by/crm:P9_consists_of ?subcreation_work_creator_with_role . ?subcreation_work_creator_with_role crm:P2_has_type ?subcreation_type_work_creator_with_role ; crm:P14_carried_out_by ?subcreation_person_work_creator_with_role . ?subcreation_type_work_creator_with_role rdfs:label ?subcreation_type_label_work_creator_with_role . ?subcreation_person_work_creat